In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import matplotlib.pyplot as plt

import numpy as np
import nltk
from nltk.corpus import stopwords
import json

In [ ]:
# df = pd.read_csv('/home/nadia/Documents/non-kuliah/CODINGCAMP/MachineLearningPlayGround/PengembanganMachineLearning/model-deployment/data/archive/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df = pd.read_csv('data/archive/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [8]:
df

,sentence,label
0,wow... loved place.,1
1,crust good.,0
2,tasty texture nasty.,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices.,1
...,...,...
995,think food flavor texture lacking.,0
996,appetite instantly gone.,0
997,overall impressed would go back.,0
998,"whole experience underwhelming, think go ninja...",0


In [7]:
# Mengubah seluruh text kedalam bentuk lowercase
df['sentence'] = df['sentence'].str.lower()
 
# Menghilangkan stopwords
stop_word = set(stopwords.words('english'))
 
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
 
# Melakukan split dataset
sentence = df['sentence'].values
label = df['label'].values
 
sentence_train, sentence_test, label_train, label_test = train_test_split(sentence, label, test_size=0.2, shuffle=False)
 
# Membuat tokenisasi
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter untuk menghilangkan symbols
 
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)
 
tokenizer.fit_on_texts(sentence_train)
 
# Menyimpan word_index kedalam sebuah file json
word_index = tokenizer.word_index
 
with open('word_index.json', 'w') as fp:
    json.dump(word_index, fp)
 
# Membuat sequences dan melakukan padding
train_sekuens = tokenizer.texts_to_sequences(sentence_train)
test_sekuens = tokenizer.texts_to_sequences(sentence_test)
 
train_padded = pad_sequences(train_sekuens,
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

<>:16: SyntaxWarning: invalid escape sequence '\]'
<>:16: SyntaxWarning: invalid escape sequence '\]'
/tmp/ipykernel_17010/899678979.py:16: SyntaxWarning: invalid escape sequence '\]'
  filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter untuk menghilangkan symbols


In [9]:
# Membuat model
model = tf.keras.Sequential([
    Embedding(2000, 20, input_length=20),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
    
# Compile model
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    
# Train model
num_epochs = 30
history = model.fit(train_padded, label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, label_test),
                    verbose=1)

/home/nadia/Documents/non-kuliah/CODINGCAMP/MachineLearningPlayGround/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-04-03 00:28:46.418411: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.5484 - loss: 0.6923 - val_accuracy: 0.2400 - val_loss: 0.7238
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5720 - loss: 0.6832 - val_accuracy: 0.2400 - val_loss: 0.7614
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.5595 - loss: 0.6810 - val_accuracy: 0.2400 - val_loss: 0.7688
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5849 - loss: 0.6678 - val_accuracy: 0.2400 - val_loss: 0.7286
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5787 - loss: 0.6592 - val_accuracy: 0.4250 - val_loss: 0.6941
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7893 - loss: 0.6169 - val_accuracy: 0.4700 - val_loss: 0.6848
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8196 - loss: 0.5191 - val_accuracy: 0.8450 - val_loss: 0.5474
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8840 - loss: 0.3881 - val_accuracy: 0.8250 - v

In [10]:
model.save("model.h5")

In [12]:
# Convert model.h5 to model
!tensorflowjs_converter --input_format=keras model.h5 tfjs_model

2025-04-03 00:30:50.075884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743615050.124231   20864 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743615050.139328   20864 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743615050.179287   20864 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743615050.179366   20864 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743615050.179385   20864 computation_placer.cc:177] computation placer alr

## Menerapkan Model pada Web Browser dengan TensorFlow.js